# construct a Bayesian network considering medical data.

In [37]:
from pgmpy.models import BayesianNetwork

cancer_model = BayesianNetwork([('Pollution', 'Cancer'), 
                              ('Smoker', 'Cancer'),
                              ('Cancer', 'Xray'),
                              ('Cancer', 'Dyspnoea')])

In [15]:
print(cancer_model)

BayesianModel with 5 nodes and 4 edges


In [16]:
cancer_model.nodes()

NodeView(('Pollution', 'Cancer', 'Smoker', 'Xray', 'Dyspnoea'))

In [17]:
cancer_model.edges()

OutEdgeView([('Pollution', 'Cancer'), ('Cancer', 'Xray'), ('Cancer', 'Dyspnoea'), ('Smoker', 'Cancer')])

In [18]:
cancer_model.get_cpds()

[]

# 7.1.2 Creation  of Conditional Probability Table 

In [19]:
# Définition des relations.
from pgmpy.factors.discrete import TabularCPD

cpd_poll = TabularCPD(variable='Pollution', variable_card=2,
                      values=[[0.9], [0.1]])
cpd_smoke = TabularCPD(variable='Smoker', variable_card=2,
                       values=[[0.3], [0.7]])
cpd_cancer = TabularCPD(variable='Cancer', variable_card=2,
                        values=[[0.03, 0.05, 0.001, 0.02],
                                [0.97, 0.95, 0.999, 0.98]],
                        evidence=['Smoker', 'Pollution'],
                        evidence_card=[2, 2])
cpd_xray = TabularCPD(variable='Xray', variable_card=2,
                      values=[[0.9, 0.2], [0.1, 0.8]],
                      evidence=['Cancer'], evidence_card=[2])
cpd_dysp = TabularCPD(variable='Dyspnoea', variable_card=2,
                      values=[[0.65, 0.3], [0.35, 0.7]],
                      evidence=['Cancer'], evidence_card=[2])

In [20]:
# Associating the parameters with the model structure.
cancer_model.add_cpds(cpd_poll, cpd_smoke, cpd_cancer, cpd_xray, cpd_dysp)

# Checking if the cpds are valid for the model.
cancer_model.check_model()

True

In [21]:
#trouver les chemins des noeuds pour determiner qu'est ce qui donne des infos sur un noeud
print(cancer_model.active_trail_nodes('Pollution'))
print(cancer_model.active_trail_nodes('Smoker'))


{'Pollution': {'Xray', 'Pollution', 'Dyspnoea', 'Cancer'}}
{'Smoker': {'Xray', 'Dyspnoea', 'Cancer', 'Smoker'}}


In [22]:
cancer_model.get_cpds()

[<TabularCPD representing P(Pollution:2) at 0x29f7fcc8bd0>,
 <TabularCPD representing P(Smoker:2) at 0x29f7f671150>,
 <TabularCPD representing P(Cancer:2 | Smoker:2, Pollution:2) at 0x29f7fccbb90>,
 <TabularCPD representing P(Xray:2 | Cancer:2) at 0x29f7fccbd50>,
 <TabularCPD representing P(Dyspnoea:2 | Cancer:2) at 0x29f7fcc91d0>]

In [23]:
print(cancer_model.get_cpds('Pollution'))

+--------------+-----+
| Pollution(0) | 0.9 |
+--------------+-----+
| Pollution(1) | 0.1 |
+--------------+-----+


In [24]:
print(cancer_model.get_cpds('Smoker'))

+-----------+-----+
| Smoker(0) | 0.3 |
+-----------+-----+
| Smoker(1) | 0.7 |
+-----------+-----+


In [25]:
print(cancer_model.get_cpds('Xray'))

+---------+-----------+-----------+
| Cancer  | Cancer(0) | Cancer(1) |
+---------+-----------+-----------+
| Xray(0) | 0.9       | 0.2       |
+---------+-----------+-----------+
| Xray(1) | 0.1       | 0.8       |
+---------+-----------+-----------+


In [26]:
print(cancer_model.get_cpds('Dyspnoea'))

+-------------+-----------+-----------+
| Cancer      | Cancer(0) | Cancer(1) |
+-------------+-----------+-----------+
| Dyspnoea(0) | 0.65      | 0.3       |
+-------------+-----------+-----------+
| Dyspnoea(1) | 0.35      | 0.7       |
+-------------+-----------+-----------+


In [27]:
print(cancer_model.get_cpds('Cancer'))

+-----------+--------------+--------------+--------------+--------------+
| Smoker    | Smoker(0)    | Smoker(0)    | Smoker(1)    | Smoker(1)    |
+-----------+--------------+--------------+--------------+--------------+
| Pollution | Pollution(0) | Pollution(1) | Pollution(0) | Pollution(1) |
+-----------+--------------+--------------+--------------+--------------+
| Cancer(0) | 0.03         | 0.05         | 0.001        | 0.02         |
+-----------+--------------+--------------+--------------+--------------+
| Cancer(1) | 0.97         | 0.95         | 0.999        | 0.98         |
+-----------+--------------+--------------+--------------+--------------+


# 7.1.4 Determining the Local independencies

In [28]:
cancer_model.local_independencies('Xray')

(Xray ⟂ Pollution, Dyspnoea, Smoker | Cancer)

In [29]:
cancer_model.local_independencies('Pollution')

(Pollution ⟂ Smoker)

In [30]:
cancer_model.local_independencies('Smoker')

(Smoker ⟂ Pollution)

In [31]:
cancer_model.local_independencies('Dyspnoea')

(Dyspnoea ⟂ Xray, Pollution, Smoker | Cancer)

In [32]:
cancer_model.local_independencies('Cancer')

In [33]:
cancer_model.get_independencies()

(Dyspnoea ⟂ Xray, Pollution, Smoker | Cancer)
(Dyspnoea ⟂ Pollution, Smoker | Xray, Cancer)
(Dyspnoea ⟂ Xray, Smoker | Pollution, Cancer)
(Dyspnoea ⟂ Xray, Pollution | Cancer, Smoker)
(Dyspnoea ⟂ Smoker | Xray, Pollution, Cancer)
(Dyspnoea ⟂ Pollution | Xray, Cancer, Smoker)
(Dyspnoea ⟂ Xray | Pollution, Cancer, Smoker)
(Smoker ⟂ Pollution)
(Smoker ⟂ Xray, Dyspnoea | Cancer)
(Smoker ⟂ Dyspnoea | Xray, Cancer)
(Smoker ⟂ Xray | Dyspnoea, Cancer)
(Smoker ⟂ Xray, Dyspnoea | Pollution, Cancer)
(Smoker ⟂ Dyspnoea | Xray, Pollution, Cancer)
(Smoker ⟂ Xray | Pollution, Dyspnoea, Cancer)
(Xray ⟂ Pollution, Dyspnoea, Smoker | Cancer)
(Xray ⟂ Dyspnoea, Smoker | Pollution, Cancer)
(Xray ⟂ Pollution, Smoker | Dyspnoea, Cancer)
(Xray ⟂ Pollution, Dyspnoea | Cancer, Smoker)
(Xray ⟂ Smoker | Pollution, Dyspnoea, Cancer)
(Xray ⟂ Dyspnoea | Pollution, Cancer, Smoker)
(Xray ⟂ Pollution | Dyspnoea, Cancer, Smoker)
(Pollution ⟂ Smoker)
(Pollution ⟂ Xray, Dyspnoea | Cancer)
(Pollution ⟂ Dyspnoea | Xray, Can